In [1]:
import pandas as pd
import json
import os

In [3]:
data = json.load(open('step3_result.json'))

In [4]:
rows = []
for issue, commits in data.items():
    for commit in commits:
        for file in commit['files']:
            if file['added_lines'] + file['deleted_lines'] + file['added_methods'] + file['deleted_method'] + file['modified_methods'] == 0:
                continue
            row = {
                'issue': issue,
                'commit_hash': commit['commit_hash'],
                'dmm_unit_size': commit['dmm_metrics']['dmm_unit_size'],
                'dmm_unit_complexity': commit['dmm_metrics']['dmm_unit_complexity'],
                'dmm_unit_interfacing': commit['dmm_metrics']['dmm_unit_interfacing'],
                'added_lines': file['added_lines'],
                'deleted_lines': file['deleted_lines'],
                'added_methods': file['added_methods'],
                'deleted_method': file['deleted_method'],
                'modified_methods': file['modified_methods'],
                'complexity': file['complexity'] if file['complexity'] else 0,
                'filename': os.path.basename(file['filename'])
            }
            rows.append(row)

In [5]:
df = pd.DataFrame(rows)

In [7]:
"""
original_excel_df:
Key	Classes
YARN-5561	False False True
"""

original_excel_df = pd.read_excel('Issues_assignment1.xlsx', sheet_name='Group2')

# split Classes column into 3 columns into Existence, Property, Executive
original_excel_df[['Existence', 'Property', 'Executive']] = original_excel_df['Classes'].str.split(' ', expand=True)

# convert True/False to labels
original_excel_df['Existence'] = original_excel_df['Existence'].map({'True': 'Existence', 'False': 'None'})
original_excel_df['Property'] = original_excel_df['Property'].map({'True': 'Property', 'False': 'None'})
original_excel_df['Executive'] = original_excel_df['Executive'].map({'True': 'Executive', 'False': 'None'})

original_excel_df.head(3)


,Key,Classes,Existence,Property,Executive
0,HADOOP-7206,False False True,None,None,Executive
1,HADOOP-7144,False False True,None,None,Executive
2,HADOOP-7109,False False True,None,None,Executive


In [8]:
df = df.merge(original_excel_df[['Key', 'Existence', 'Property', 'Executive']], left_on='issue', right_on='Key', how='left')
df = df.drop(columns=['Key'])

In [9]:
df.head(3)

,issue,commit_hash,dmm_unit_size,dmm_unit_complexity,dmm_unit_interfacing,added_lines,deleted_lines,added_methods,deleted_method,modified_methods,complexity,filename,Existence,Property,Executive
0,HADOOP-7206,7e1e4bf50fa83083e762fc267b5215d606a64c3e,0.684411,0.684411,0.585551,2,0,0,0,0,0,CHANGES.txt,None,None,Executive
1,HADOOP-7206,7e1e4bf50fa83083e762fc267b5215d606a64c3e,0.684411,0.684411,0.585551,4,0,0,0,0,0,ivy.xml,None,None,Executive
2,HADOOP-7206,7e1e4bf50fa83083e762fc267b5215d606a64c3e,0.684411,0.684411,0.585551,5,0,0,0,0,0,hadoop-common-template.xml,None,None,Executive


In [11]:
with pd.ExcelWriter('Step4.xlsx', mode='a', if_sheet_exists='overlay') as writer:
    df.to_excel(writer, sheet_name='Data', index=False)